In [24]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from InterleavedGCNN import InterleavedGCNN
from GraphDataset import GraphDataset
from train_val_test import train
from parentBCELoss import parentBCELoss

#### Parent ILP

In [2]:
# Load dataset
train_dir = f'./../dataset/training/parent_graphs'
train_data = GraphDataset(root=train_dir, data_list=[])
parents_loader = DataLoader(train_data, batch_size=50, shuffle=True)

#### Children ILP

In [3]:
# Load dataset
train_dir = f'./../dataset/training/children_graphs'
train_data = GraphDataset(root=train_dir, data_list=[])
children_loader = DataLoader(train_data, batch_size=50, shuffle=True)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hidden_channels = 64
num_layers = 3

In [5]:
model = InterleavedGCNN(
    var_in_channels=children_loader.dataset[0]["var_nodes"].x.shape[1],
    cons_in_channels=children_loader.dataset[0]["constr_nodes"].x.shape[1],
    hidden_channels=hidden_channels,
    num_layers=num_layers,
    edge_attr_dim=1
).to(device)

# Define loss function and optimizer
criterion = {"children": nn.BCEWithLogitsLoss(),
                "parents": parentBCELoss()}
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Training and validation
num_epochs = 10
train_loss = []
val_loss = []

In [6]:
for epoch in range(1, num_epochs + 1):
    losses = train(model = model, children_loader = children_loader, 
                    parents_loader = parents_loader, 
                    children_criterion = criterion["children"],
                    parents_criterion = criterion["parents"],
                    optimizer = optimizer, device = device)
    train_loss.append(losses[0])
    val_loss.append(losses[1])
    print(f"Epoch: {epoch:02d}, Train Loss: {losses[0]:.4f}, Val Loss: {losses[1]:.4f}")

IndexError: too many indices for tensor of dimension 0

In [42]:
var = torch.tensor(np.random.uniform(0, 1, size=500), dtype = torch.float64)
var.prod()

tensor(7.5661e-219, dtype=torch.float64)

In [47]:
data = parents_loader.dataset[0]
data["var_nodes"].prob, data["var_nodes"].prob_divisor, data["var_nodes"].prob_dividend

(tensor([0.0049]), tensor([0.3679]), tensor([75.1569]))